# Data analysis for ITSC

Test cases are chipped apart in 3 computers: HOME (nreq = 10, 20), TU (nreq - 50, 100) and SERVER (nreq - 200)

Number of vehicles is #v  = {5, 10, 20, 50}

Data from different computers is joined, and only results where #R < #V are considered.

In [13]:
# If you're working with a notebook, don't forget to use Matplotlib magic! 
%matplotlib inline

# Import `pandas` under the alias `pd`
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

# Import `numpy` under the alias `np`
import numpy as np

# Import `seaborn` under the alias `sns`
import seaborn as sns

# Set the Seaborn theme if desired
sns.set_style('darkgrid')

root = 'C:/Users/breno/OneDrive/Phd_TU/PROJECTS/STMAASSL2018/STMAASSL/output/'

input_files = ["results_HOME.csv", "results_TUDPC.csv", "results_UHH.csv", "results_TUDSRV.csv"]

# Load results from all computers
# HOME
all_df = [pd.read_csv(root+f_name) for f_name in input_files]

# Overall table
overall_dt = pd.concat(all_df)

solution_id = ["REGION",
               "SUBNETWORK_TYPES",
               "SPREAD",
               "#ZONES",
               "#TEST",
               "PRICE",
               "#VEHICLES",
               "COMPARTMENTS DIV.",
               "#REQUESTS",
               "DEMAND_DIST_MODE",
               "SL_SHARE",
               "TIME_HORIZON",
               "TRIPS_DIST",
               "DEMAND_LIMIT"]

overall_dt.drop_duplicates(subset=solution_id)

demand_label = {"HI":"Low", "LO":"High", "AV":"Moderate"}

overall_dt["DEMAND_DIST_MODE"] = overall_dt["DEMAND_DIST_MODE"].map(lambda n:demand_label[n])

# Set Categorical order to DEMAND_DIST_MODE
overall_dt["DEMAND_DIST_MODE"] = pd.Categorical(overall_dt["DEMAND_DIST_MODE"], ["Low", "Moderate", "High"])


         
        


# Read as string (e.g. "04") and no index_col - ,  dtype=object, index_col=False
# Saving
#df_10_20.to_csv("C:/Users/breno/OneDrive/Phd_TU/PROJECTS/STMAASSL2018/STMAASSL/results_clean1.csv", index=False)
#df_50_100.to_csv("C:/Users/breno/OneDrive/Phd_TU/PROJECTS/STMAASSL2018/STMAASSL/results_clean2.csv", index=False)


# Clean lines where #vehicles > #requests
# df = df[df["#VEHICLES"]<=df["#REQUESTS"]]



#Select columns
df = overall_dt[[
         "PRICE",
         "#ZONES",
         "SPREAD",
         "#TEST",
         "#VEHICLES",
         "A",
         "C",
         "D",
         "OPERATING_VEHICLES",
         "#REQUESTS",
        "#ATTENDED",
         "DEMAND_DIST_MODE",
         "TIME_HORIZON",
         "OVERALL_OCCUPANCY",
         "PROFIT",
         "REQUESTS_REVENUE",
         "OPERATIONAL_COSTS",
         "SOL_GAP",
         "OBJ_VAL",
         "PREPROCESS",
         "RUNTIME"]]

# LABELS



newcols = {          "SPREAD" : "AD Coverage",
                     "#ZONES" : "#Zones",
                  "#VEHICLES" : "#Vehicles",
                  "#ATTENDED" : "#Attended",
                  "#REQUESTS" : "#Requests",
           "DEMAND_DIST_MODE" : "Zone crossing",
               "TIME_HORIZON" : "Time horizon",
                 "TRIPS_DIST" : "Max. trip distance",
          "OVERALL_OCCUPANCY" : "Avg. occupancy",
         "OPERATING_VEHICLES" : "Avg. vehicles",
                     "PROFIT" : "Avg. profit",
           "REQUESTS_REVENUE" : "Avg. revenue",
          "OPERATIONAL_COSTS" : "Avg. op. cost",
                    "SOL_GAP" : "Avg. gap",
                    "OBJ_VAL" : "Objective value",
                      "PRICE" : "Cost scenario",
                    "RUNTIME" : "Runtime(s)"}

# Sort
df = df.sort_values(by=["PRICE",
                        "SPREAD",
                        "#ZONES",
                        "#VEHICLES",
                        "#REQUESTS",
                        "DEMAND_DIST_MODE",
                        "TIME_HORIZON",
                        "#TEST"])


## Separating subnetworks

**Zones:** Maps can have 1, 2, 4 and 8 zones with 10%, 25% and 50% coverage.

**Subnetworks:** Maps can have subnetworks with 10%, 25% and 50% coverage.

In [14]:
# Results zones
df_zones = df#[df["SUBNETWORK_TYPES"] == "zones"]

# Transforms Z01 to 1
df_zones["#ZONES"]=df_zones["#ZONES"].map(lambda n:int(n[1:]))

# Transforms S025 to 25%
df_zones["SPREAD"]=df_zones["SPREAD"].map(lambda n:"{}%".format(int(n[1:])))

# Transforms number of vehicles

df_zones["#VEHICLES"]=df_zones["#VEHICLES"].map(lambda n:3*n)


df_zones["A"] = df_zones["A"] / df_zones["OPERATING_VEHICLES"]
df_zones["C"] = df_zones["C"] / df_zones["OPERATING_VEHICLES"]
df_zones["D"] = df_zones["D"] / df_zones["OPERATING_VEHICLES"]
df_zones["OPERATING_VEHICLES"] = df_zones["OPERATING_VEHICLES"] / df_zones["#VEHICLES"]
df_zones["#ATTENDED"] = df_zones["#ATTENDED"] / df_zones["#REQUESTS"]

# Results subnetworks
#df_subnetworks = df[df["SUBNETWORK_TYPES"] == "subnetworks"]

df_zones

,PRICE,#ZONES,SPREAD,#TEST,#VEHICLES,A,C,D,OPERATING_VEHICLES,#REQUESTS,...,DEMAND_DIST_MODE,TIME_HORIZON,OVERALL_OCCUPANCY,PROFIT,REQUESTS_REVENUE,OPERATIONAL_COSTS,SOL_GAP,OBJ_VAL,PREPROCESS,RUNTIME
961,S01,1,10%,1,15,0.375000,0.375000,0.250000,0.533333,10,...,Low,01min,37.454660,76.46,84.422,7.963,0.000000,76.459,0.0,0.140400
385,S01,1,10%,2,15,0.400000,0.400000,0.200000,0.666667,10,...,Low,01min,55.877229,107.60,118.838,11.239,0.000000,107.599,0.0,0.070000
217,S01,1,10%,3,15,0.333333,0.444444,0.222222,0.600000,10,...,Low,01min,54.150355,101.08,109.494,8.410,0.000000,101.084,0.0,0.060005
25,S01,1,10%,4,15,0.250000,0.625000,0.125000,0.533333,10,...,Low,01min,47.977267,94.42,102.408,7.983,0.000000,94.425,0.0,0.139000
361,S01,1,10%,5,15,0.142857,0.428571,0.428571,0.466667,10,...,Low,01min,44.664598,70.90,81.263,10.367,0.000000,70.896,0.0,0.100010
784,S01,1,10%,1,15,0.333333,0.444444,0.222222,0.600000,10,...,Low,05min,48.278189,84.88,96.866,11.990,0.000000,84.876,0.0,0.208138
1211,S01,1,10%,2,15,0.333333,0.555556,0.111111,0.600000,10,...,Low,05min,33.940013,86.53,94.628,8.097,0.000000,86.531,0.0,0.237181
1138,S01,1,10%,3,15,0.375000,0.500000,0.125000,0.533333,10,...,Low,05min,51.744774,105.62,115.971,10.354,0.000000,105.617,0.0,0.140104
1201,S01,1,10%,4,15,0.285714,0.571429,0.142857,0.466667,10,...,Low,05min,38.976476,76.44,83.466,7.025,0.000000,76.441,0.0,0.215141
915,S01,1,10%,5,15,0.250000,0.625000,0.125000,0.533333,10,...,Low,05min,31.492662,70.68,78.641,7.965,0.000000,70.676,0.0,0.272692


## Grouping test cases
Since 10 test cases are generated for each zone configuration (e.g., *delft-the-netherlands_zones_S025_Z08_01, delft-the-netherlands_zones_S025_Z08_02, ..., delft-the-netherlands_zones_S025_Z08_10*), we get the mean for every output from a single configuration (e.g., *delft-the-netherlands_zones_S025_Z08*).

In [15]:
"""
df_zones_vehicle = df_zones.groupby(["#Zones", "#Requests"], as_index = False)["Avg. vehicles"].mean()
g_df_zones_vehicle = sns.FacetGrid(df_zones_vehicle, col="#Zones", hue="#Zones")
g_df_zones_vehicle.map(sns.barplot, "#Requests", "Avg. vehicles")
g_df_zones_vehicle.fig.subplots_adjust(wspace=.2)"""

'\ndf_zones_vehicle = df_zones.groupby(["#Zones", "#Requests"], as_index = False)["Avg. vehicles"].mean()\ng_df_zones_vehicle = sns.FacetGrid(df_zones_vehicle, col="#Zones", hue="#Zones")\ng_df_zones_vehicle.map(sns.barplot, "#Requests", "Avg. vehicles")\ng_df_zones_vehicle.fig.subplots_adjust(wspace=.2)'

In [16]:


avg_data_tests = {"OPERATING_VEHICLES" : 'mean',
                             "A" : 'mean',
                             "C" : 'mean',
                             "D" : 'mean',
                     "#ATTENDED" : 'mean',
                        "PROFIT" : 'mean',
              "REQUESTS_REVENUE" : 'mean',
             "OPERATIONAL_COSTS" : 'mean',
             "OVERALL_OCCUPANCY" : 'mean',
                       "SOL_GAP" : 'mean',
                       "OBJ_VAL" : 'mean',
                    "PREPROCESS" : 'mean',
                       "RUNTIME" : 'mean',
                         "#TEST" : 'count'}

decimal_cases = {"OVERALL_OCCUPANCY" : 2,
                            "PROFIT" : 2,
                  "REQUESTS_REVENUE" : 2,
                 "OPERATIONAL_COSTS" : 2,
                           "SOL_GAP" : 5,
                           "OBJ_VAL" : 2,
                        "PREPROCESS" : 2}

#df_zones_mean.rename(columns=newcols, inplace=True)

#df_zones_mean.to_csv("C:/Users/breno/OneDrive/Phd_TU/PROJECTS/STMAASSL2018/STMAASSL/a.csv")

# Show
df_zones_mean_tests = df_zones.groupby([
                   "PRICE",
                   "#ZONES",
                   "SPREAD",
                   "#VEHICLES",
                   "#REQUESTS",
                   "DEMAND_DIST_MODE",
                   "TIME_HORIZON"], as_index=False).agg(avg_data_tests)


avg_data_zones = {"OPERATING_VEHICLES" : 'mean',
                             "A" : 'mean',
                             "C" : 'mean',
                             "D" : 'mean',
                     "#ATTENDED" : 'mean',
                        "PROFIT" : 'mean',
              "REQUESTS_REVENUE" : 'mean',
             "OPERATIONAL_COSTS" : 'mean',
             "OVERALL_OCCUPANCY" : 'mean',
                       "SOL_GAP" : 'mean',
                       "OBJ_VAL" : 'mean',
                    "PREPROCESS" : 'mean',
                       "RUNTIME" : 'mean',
                        "#ZONES" : 'count'}

# Show
df_zones_mean_zones = df_zones_mean_tests.groupby([
                   "PRICE",
                   "SPREAD",
                   "#VEHICLES",
                   "#REQUESTS",
                   "DEMAND_DIST_MODE",
                   "TIME_HORIZON"], as_index=False).agg(avg_data_zones)


df2 = pd.DataFrame(df_zones_mean_tests)

# Rounded dataframe
df2.round(decimal_cases)

#Rename columns
#df2.rename(columns=newcols, inplace=True)


df_zones_mean_zones
#df2
#df2.round(decimal_cases)
#df_zones_mean_show.head(1000)


,PRICE,SPREAD,#VEHICLES,#REQUESTS,DEMAND_DIST_MODE,TIME_HORIZON,OPERATING_VEHICLES,A,C,D,#ATTENDED,PROFIT,REQUESTS_REVENUE,OPERATIONAL_COSTS,OVERALL_OCCUPANCY,SOL_GAP,OBJ_VAL,PREPROCESS,RUNTIME,#ZONES
0,S01,10%,15,10,Low,01min,0.595556,0.331376,0.405423,0.263201,0.986667,85.753333,96.804200,11.050933,46.506770,0.000000e+00,85.753267,0.000000,0.096937,3
1,S01,10%,15,10,Low,05min,0.560926,0.309788,0.498589,0.191623,0.991667,81.868194,92.638981,10.771872,42.296080,0.000000e+00,81.867108,0.000000,0.185832,3
2,S01,10%,15,10,Low,10min,0.564259,0.356129,0.424675,0.219196,1.000000,88.640750,100.235303,11.594758,46.718938,0.000000e+00,88.640544,0.000000,0.167056,3
3,S01,10%,15,10,Low,20min,0.522646,0.352809,0.407458,0.239733,1.000000,88.946159,100.583316,11.637567,48.366674,0.000000e+00,88.945749,0.000000,0.169063,3
4,S01,10%,15,10,Moderate,01min,0.622222,0.323148,0.376296,0.300556,0.960000,83.710667,95.869933,12.160467,45.762503,0.000000e+00,83.709467,0.000000,0.089257,3
5,S01,10%,15,10,Moderate,05min,0.609074,0.270309,0.393488,0.336204,0.983333,83.088194,97.366939,14.277783,45.075984,0.000000e+00,83.089156,0.000000,0.154432,3
6,S01,10%,15,10,Moderate,10min,0.599444,0.284729,0.424570,0.290701,1.000000,80.715861,93.131625,12.416675,42.810469,0.000000e+00,80.714950,0.000000,0.183446,3
7,S01,10%,15,10,Moderate,20min,0.504021,0.288555,0.374118,0.337327,0.993333,90.300222,103.812063,13.511017,50.044554,7.222222e-07,90.301046,0.047619,0.186349,3
8,S01,10%,15,10,High,01min,0.564444,0.190000,0.313519,0.496481,0.880000,77.620667,91.439867,13.820067,48.527194,0.000000e+00,77.619800,0.000000,0.082871,3
9,S01,10%,15,10,High,05min,0.570000,0.165448,0.331537,0.503016,0.893056,87.901278,101.911272,14.009561,52.775339,0.000000e+00,87.901711,0.000000,0.132215,3


In [17]:
demand_dist = df_zones_mean_zones[df_zones_mean_zones["PRICE"]=="S03"]

demand_dist = demand_dist[["#VEHICLES", "SPREAD", "#REQUESTS", "DEMAND_DIST_MODE", "TIME_HORIZON", "A", "C", "D", "#ATTENDED", "OPERATING_VEHICLES", "RUNTIME"]]

demand_dist

        

,#VEHICLES,SPREAD,#REQUESTS,DEMAND_DIST_MODE,TIME_HORIZON,A,C,D,#ATTENDED,OPERATING_VEHICLES,RUNTIME
576,15,10%,10,Low,01min,0.340635,0.357275,0.302090,0.993333,0.604444,0.096719
577,15,10%,10,Low,05min,0.336085,0.411243,0.252672,1.000000,0.568889,0.183113
578,15,10%,10,Low,10min,0.357275,0.396164,0.246561,1.000000,0.560000,0.170790
579,15,10%,10,Low,20min,0.376402,0.371587,0.252011,1.000000,0.528889,0.169227
580,15,10%,10,Moderate,01min,0.327593,0.337778,0.334630,0.966667,0.626667,0.090480
581,15,10%,10,Moderate,05min,0.267037,0.373333,0.359630,0.993333,0.613333,0.153094
582,15,10%,10,Moderate,10min,0.310741,0.347407,0.341852,1.000000,0.582222,0.179571
583,15,10%,10,Moderate,20min,0.291429,0.353122,0.355450,1.000000,0.524444,0.173315
584,15,10%,10,High,01min,0.188148,0.305000,0.506852,0.886667,0.568889,0.075919
585,15,10%,10,High,05min,0.169868,0.303968,0.526164,0.913333,0.577778,0.132044


In [18]:
demand_agg = {    "TIME_HORIZON" : 'count',
                             "A" : 'mean',
                             "C" : 'mean',
                             "D" : 'mean',
                     "#ATTENDED" : 'mean',
            "OPERATING_VEHICLES" : 'mean',
                     "#REQUESTS" : 'count',
                     "#VEHICLES" : 'count',
                       "RUNTIME" : 'mean'}

sub_demand_dist = demand_dist.groupby([
                    "SPREAD",
                    "DEMAND_DIST_MODE"], as_index=False).agg(demand_agg)


table_demand_ACD = sub_demand_dist[["SPREAD", "DEMAND_DIST_MODE", "A", "C", "D", "RUNTIME"]]

#table_demand_ACD.sort_index(inplace=True)

table_demand_ACD

,SPREAD,DEMAND_DIST_MODE,A,C,D,RUNTIME
0,10%,Low,0.358647,0.391033,0.250321,23.152855
1,10%,Moderate,0.301334,0.371472,0.327193,23.594466
2,10%,High,0.170528,0.334878,0.494594,22.248976
3,25%,Low,0.390872,0.328349,0.280779,18.990266
4,25%,Moderate,0.342545,0.297337,0.360118,18.791090
5,25%,High,0.232098,0.227836,0.540065,11.320318
6,50%,Low,0.416192,0.294105,0.289702,11.001811
7,50%,Moderate,0.392793,0.271350,0.335857,8.359230
8,50%,High,0.323357,0.170315,0.506328,12.115431


In [19]:
table_demand_ACD["A"] = table_demand_ACD["A"].map(lambda n:"{0:.2f}%".format(n*100))
table_demand_ACD["C"] = table_demand_ACD["C"].map(lambda n:"{0:.2f}%".format(n*100))
table_demand_ACD["D"] = table_demand_ACD["D"].map(lambda n:"{0:.2f}%".format(n*100))
table_demand_ACD["RUNTIME"] = table_demand_ACD["RUNTIME"].map(lambda n:round(n, 2))

table_demand_ACD.rename(columns=newcols, inplace=True)

table_demand_ACD= table_demand_ACD.set_index([newcols["SPREAD"], newcols["DEMAND_DIST_MODE"]], drop=True)

table_demand_ACD

A       C       D  Runtime(s)
AD Coverage Zone crossing                                    
10%         Low            35.86%  39.10%  25.03%       23.15
            Moderate       30.13%  37.15%  32.72%       23.59
            High           17.05%  33.49%  49.46%       22.25
25%         Low            39.09%  32.83%  28.08%       18.99
            Moderate       34.25%  29.73%  36.01%       18.79
            High           23.21%  22.78%  54.01%       11.32
50%         Low            41.62%  29.41%  28.97%       11.00
            Moderate       39.28%  27.13%  33.59%        8.36
            High           32.34%  17.03%  50.63%       12.12

In [20]:
# Latex
print(table_demand_ACD.to_latex(multicolumn=True, multirow=True))

\begin{tabular}{lllllr}
\toprule
    &     &       A &       C &       D &  Runtime(s) \\
AD Coverage & Zone crossing &         &         &         &             \\
\midrule
\multirow{3}{*}{10\%} & Low &  35.86\% &  39.10\% &  25.03\% &       23.15 \\
    & Moderate &  30.13\% &  37.15\% &  32.72\% &       23.59 \\
    & High &  17.05\% &  33.49\% &  49.46\% &       22.25 \\
\cline{1-6}
\multirow{3}{*}{25\%} & Low &  39.09\% &  32.83\% &  28.08\% &       18.99 \\
    & Moderate &  34.25\% &  29.73\% &  36.01\% &       18.79 \\
    & High &  23.21\% &  22.78\% &  54.01\% &       11.32 \\
\cline{1-6}
\multirow{3}{*}{50\%} & Low &  41.62\% &  29.41\% &  28.97\% &       11.00 \\
    & Moderate &  39.28\% &  27.13\% &  33.59\% &        8.36 \\
    & High &  32.34\% &  17.03\% &  50.63\% &       12.12 \\
\bottomrule
\end{tabular}



In [21]:
#DF mean tests

table_demand_ACD.head()

A       C       D  Runtime(s)
AD Coverage Zone crossing                                    
10%         Low            35.86%  39.10%  25.03%       23.15
            Moderate       30.13%  37.15%  32.72%       23.59
            High           17.05%  33.49%  49.46%       22.25
25%         Low            39.09%  32.83%  28.08%       18.99
            Moderate       34.25%  29.73%  36.01%       18.79

In [22]:
op_cost = df_zones_mean_zones[["#VEHICLES", "PRICE", "#REQUESTS", "#ATTENDED", "A", "C", "D", "OPERATING_VEHICLES", "OPERATIONAL_COSTS"]]

grouped = op_cost.groupby([
                   "#VEHICLES",
                   "#REQUESTS",
                    "PRICE"], as_index=False).mean()


grouped["A"] = grouped["A"].map(lambda n:"{0:.2f}%".format(n*100))
grouped["C"] = grouped["C"].map(lambda n:"{0:.2f}%".format(n*100))
grouped["D"] = grouped["D"].map(lambda n:"{0:.2f}%".format(n*100))
grouped["OPERATING_VEHICLES"] = grouped["OPERATING_VEHICLES"].map(lambda n:"{0:.2f}%".format(n*100))
grouped["#ATTENDED"] = grouped["#ATTENDED"].map(lambda n:"{0:.2f}%".format(n*100))
grouped["OPERATIONAL_COSTS"] = grouped["OPERATIONAL_COSTS"].map(lambda n:round(n, 2))

grouped.rename(columns=newcols, inplace=True)

grouped = grouped.set_index([newcols["#VEHICLES"], newcols["#REQUESTS"], newcols["PRICE"]], drop=True)

grouped

#Attended       A       C       D  \
#Vehicles #Requests Cost scenario                                     
15        10        S01              96.39%  30.18%  30.65%  39.17%   
                    S02              97.09%  31.02%  29.42%  39.56%   
                    S03              97.28%  32.18%  27.73%  40.09%   
          20        S01              83.95%  32.00%  29.11%  38.89%   
                    S02              84.37%  32.49%  28.75%  38.77%   
                    S03              84.49%  32.77%  28.40%  38.83%   
          40        S01              59.32%  34.29%  30.39%  35.32%   
                    S02              53.75%  34.35%  30.32%  35.33%   
                    S03              42.13%  34.36%  30.05%  35.59%   
30        10        S01              99.53%  30.53%  32.61%  36.87%   
                    S02              99.74%  31.38%  31.88%  36.74%   
                    S03              99.80%  32.40%  29.89%  37.72%   
          20        S01              98.27%  30.01%  31.93%  38.06%   
                    S02              98.82%  30.89%  31.13%  37.97%   
                    S03              98.96%  31.78%  29.53%  38.69%   
          40        S01              87.85%  32.76%  29.14%  38.10%   
                    S02              84.04%  32.43%  29.97%  37.61%   
                    S03              75.37%  33.48%  28.30%  38.22%   
60        10        S01              99.88%  31.56%  34.11%  34.33%   
                    S02              99.95%  32.36%  33.81%  33.83%   
                    S03             100.00%  33.39%  31.96%  34.65%   
          20        S01              99.76%  30.74%  34.39%  34.87%   
                    S02              99.93%  31.72%  33.84%  34.44%   
                    S03              99.96%  32.59%  32.26%  35.15%   
          40        S01              98.90%  30.48%  33.07%  36.46%   
                    S02              98.69%  30.73%  32.87%  36.40%   
                    S03              97.85%  31.07%  30.69%  38.23%   

                                  Avg. vehicles  Avg. op. cost  
#Vehicles #Requests Cost scenario                               
15        10        S01                  56.36%          13.72  
                    S02                  56.88%          11.45  
                    S03                  56.84%           8.93  
          20        S01                  84.22%          23.80  
                    S02                  84.46%          19.67  
                    S03                  84.77%          15.32  
          40        S01                  94.96%          31.81  
                    S02                  94.79%          23.25  
                    S03                  94.22%          13.67  
30        10        S01                  30.55%          12.65  
                    S02                  30.63%          10.46  
                    S03                  30.62%           8.15  
          20        S01                  55.31%          25.41  
                    S02                  55.65%          21.24  
                    S03                  55.79%          16.60  
          40        S01                  83.62%          46.62  
                    S02                  85.57%          36.62  
                    S03                  87.68%          24.51  
60        10        S01                  15.81%          11.56  
                    S02                  15.83%           9.54  
                    S03                  15.86%           7.48  
          20        S01                  30.10%          23.33  
                    S02                  30.14%          19.31  
                    S03                  30.21%          15.11  
          40        S01                  54.58%          48.80  
                    S02                  55.99%          41.08  
                    S03                  59.36%          31.06

In [23]:
# Getting latex table
print(grouped.to_latex(multicolumn=True, multirow=True))

\begin{tabular}{llllllllr}
\toprule
   &    &     & \#Attended &       A &       C &       D & Avg. vehicles &  Avg. op. cost \\
\#Vehicles & \#Requests & Cost scenario &           &         &         &         &               &                \\
\midrule
\multirow{9}{*}{15} & \multirow{3}{*}{10} & S01 &    96.39\% &  30.18\% &  30.65\% &  39.17\% &        56.36\% &          13.72 \\
   &    & S02 &    97.09\% &  31.02\% &  29.42\% &  39.56\% &        56.88\% &          11.45 \\
   &    & S03 &    97.28\% &  32.18\% &  27.73\% &  40.09\% &        56.84\% &           8.93 \\
\cline{2-9}
   & \multirow{3}{*}{20} & S01 &    83.95\% &  32.00\% &  29.11\% &  38.89\% &        84.22\% &          23.80 \\
   &    & S02 &    84.37\% &  32.49\% &  28.75\% &  38.77\% &        84.46\% &          19.67 \\
   &    & S03 &    84.49\% &  32.77\% &  28.40\% &  38.83\% &        84.77\% &          15.32 \\
\cline{2-9}
   & \multirow{3}{*}{40} & S01 &    59.32\% &  34.29\% &  30.39\% &  35.32\% &        9

In [24]:
g = sns.factorplot(x="#Requests", y="#Attended",
                    hue="Cost scenario", col="#Vehicles",
                    data=grouped, kind="bar",
                    size=4, aspect=.7)

ValueError: Could not interpret input '#Requests'

In [ ]:
grouped2 = grouped[["#VEHICLES", "PRICE", "#REQUESTS", "OPERATIONAL_COSTS", "A", "C", "D"]]

pivoted_op_costs = df.pivot_table("OPERATIONAL_COSTS", ["#VEHICLES", "PRICE"], "#REQUESTS")
pivoted_op_costs

In [ ]:
pivoted_v_share = df.pivot_table(["A", "C", "D"], ["#VEHICLES", "PRICE"], "#REQUESTS")
pivoted_v_share

In [ ]:
sns.pairplot(data=df_zones_mean_tests[["OPERATING_VEHICLES", "#REQUESTS", "#ZONES"]], hue="#ZONES", dropna=True)

In [ ]:
g = sns.FacetGrid(df_zones_mean_zones, col="PRICE", hue="SPREAD")
g.map(sns.regplot, "#VEHICLES", "#REQUESTS")
g.fig.subplots_adjust(wspace=.2)